<a href="https://colab.research.google.com/github/Data-Science-and-Data-Analytics-Courses/MITx---Machine-Learning-with-Python-From-Linear-Models-to-Deep-Learning-Jun-11-2019/blob/master/Unit%201%20Linear%20Classifiers%20and%20Generalizations%20(2%20weeks)/Project%201%3A%20Automatic%20Review%20Analyzer/Project1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Packages and libraries

In [1]:
import sys

url = "https://github.com/Data-Science-and-Data-Analytics-Courses/Notebook-Library"

!git clone "{url}" "/nblib"
%run "/nblib/.Importable.ipynb"
sys.path.append("/")

Cloning into '/nblib'...
remote: Enumerating objects: 26, done.
remote: Counting objects: 100% (26/26), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 26 (delta 12), reused 7 (delta 2), pack-reused 0
Unpacking objects: 100% (26/26), done.


In [0]:
URL = "https://github.com/Data-Science-and-Data-Analytics-Courses/MITx---Machine-Learning-with-Python-From-Linear-Models-to-Deep-Learning-Jun-11-2019"
nbdir_rel = ""

REPO = Git.clone(URL)
NBDIR = REPO/nbdir_rel
os.chdir(NBDIR)
sys.path.append("{NBDIR.as_posix()}")

In [0]:
from google.colab import drive
from nblib import Git, File

from pathlib import Path
import numpy as np

# 1. Hinge Loss


In [0]:
def hinge_loss_single(feature_vector, label, theta, theta_0):
    """
    Finds the hinge loss on a single data point given specific classification
    parameters.

    Args:
        feature_vector - A numpy array describing the given data point.
        label - A real valued number, the correct classification of the data
            point.
        theta - A numpy array describing the linear classifier.
        theta_0 - A real valued number representing the offset parameter.


    Returns: A real number representing the hinge loss associated with the
    given data point and parameters.
    """
    # Your code here
    pred = np.dot(theta, feature_vector) + theta_0
    loss = max(0, 1-label*pred)
    
    return loss
  
feature_vector = np.array([1, 2])
label, theta, theta_0 = 1, np.array([-1, 1]), -0.2
exp_res = 1 - 0.8
assert exp_res == hinge_loss_single(feature_vector, label, theta, theta_0)